# Taxi-v3 - Analyse d'agents

## 1. Imports et création de l'environnement


In [ ]:
import gymnasium as gym
import numpy as np
import random

env = gym.make("Taxi-v3")
state, info = env.reset(seed=42)
print("State initial :", state)
print("Action space :", env.action_space)
print("Observation space :", env.observation_space)


: 

## 2. Visualisation de l'environnement


In [ ]:
env.render()


## 3. Agent aléatoire (quelques épisodes)


In [ ]:
num_episodes = 5

for ep in range(num_episodes):
    state, info = env.reset()
    done = False
    total_reward = 0
    steps = 0

    print(f"\n=== Episode {ep+1} ===")
    while not done:
        action = env.action_space.sample()
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        total_reward += reward
        steps += 1

        state = next_state

    print(f"Steps: {steps}, Total reward: {total_reward}")


## 4. Agent aléatoire instrumenté (baseline statistique)


In [ ]:
import pandas as pd

num_episodes = 1000

episode_rewards = []
episode_steps = []
episode_success = []

for ep in range(num_episodes):
    state, info = env.reset()
    done = False
    total_reward = 0
    steps = 0
    success = 0

    while not done:
        action = env.action_space.sample()
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        total_reward += reward
        steps += 1

        # Si l'épisode est terminé par "terminé" (et pas juste par max-steps),
        # on considère que le taxi a réussi (dans Taxi-v3, fin normale = dropoff réussi).
        if terminated:
            success = 1

        state = next_state

    episode_rewards.append(total_reward)
    episode_steps.append(steps)
    episode_success.append(success)

random_stats = pd.DataFrame({
    "reward": episode_rewards,
    "steps": episode_steps,
    "success": episode_success,
})

random_stats.head()


In [ ]:
print("Episodes           :", num_episodes)
print("Reward moyen       :", random_stats["reward"].mean())
print("Steps moyens       :", random_stats["steps"].mean())
print("Taux de succès (%) :", 100 * random_stats["success"].mean())


## 5. Agent Q-learning (mise en place)


In [ ]:
# Hyperparamètres Q-learning
alpha = 0.1          # learning rate
gamma = 0.99         # discount factor
epsilon = 1.0        # exploration initiale
epsilon_min = 0.1
epsilon_decay = 0.999
num_episodes_train = 20000

n_states = env.observation_space.n
n_actions = env.action_space.n

Q = np.zeros((n_states, n_actions))

def choose_action(state, epsilon):
    if np.random.rand() < epsilon:
        return env.action_space.sample()      # exploration
    else:
        return np.argmax(Q[state])           # exploitation

training_rewards = []

for ep in range(num_episodes_train):
    state, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = choose_action(state, epsilon)

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Q-learning update
        best_next_action = np.argmax(Q[next_state])
        td_target = reward + gamma * Q[next_state, best_next_action] * (0 if done else 1)
        td_error = td_target - Q[state, action]
        Q[state, action] += alpha * td_error

        total_reward += reward
        state = next_state

    # decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    training_rewards.append(total_reward)

len(training_rewards), epsilon


## 6. Évaluation de l’agent Q-learning


In [ ]:
num_episodes_eval = 1000

q_rewards = []
q_steps = []
q_success = []

for ep in range(num_episodes_eval):
    state, info = env.reset()
    done = False
    total_reward = 0
    steps = 0
    success = 0

    while not done:
        # action = argmax Q[state] (politique déterministe, pas d'exploration)
        action = np.argmax(Q[state])

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        total_reward += reward
        steps += 1

        if terminated:
            success = 1

        state = next_state

    q_rewards.append(total_reward)
    q_steps.append(steps)
    q_success.append(success)

q_stats = pd.DataFrame({
    "reward": q_rewards,
    "steps": q_steps,
    "success": q_success,
})

print("Episodes (Q-learning)   :", num_episodes_eval)
print("Reward moyen (Q)        :", q_stats['reward'].mean())
print("Steps moyens (Q)        :", q_stats['steps'].mean())
print("Taux de succès Q (%)    :", 100 * q_stats['success'].mean())


## 7. Visualisation de l'apprentissage (Q-learning)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

window = 100
smoothed_rewards = np.convolve(training_rewards, np.ones(window)/window, mode="valid")

plt.figure(figsize=(8,4))
plt.plot(smoothed_rewards)
plt.xlabel(f"Épisode (moyenne glissante sur {window})")
plt.ylabel("Reward moyen")
plt.title("Évolution du reward pendant l'entraînement (Q-learning)")
plt.grid(True)
plt.show()


## 8. Où l'agent reste inefficace


In [ ]:
# Top 10 des épisodes Q-learning les plus longs
q_stats.sort_values("steps", ascending=False).head(10)


In [ ]:
# Rejouer un épisode Q-learning long pour inspecter la trajectoire
state, info = env.reset()
done = False
steps = 0

print("Épisode Q-learning (inspection)")
env.render()

while not done and steps < 50:  # limite d'affichage
    action = np.argmax(Q[state])
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    steps += 1

    print(f"Step {steps} | Action: {action} | Reward: {reward}")
    env.render()

    state = next_state


Même avec 100 % de succès, certains épisodes restent relativement longs ou passent par des trajectoires peu efficaces.
Cela montre que la performance brute (succès / échec) ne suffit pas à juger la qualité d'un agent : 
il faut aussi regarder la structure des décisions, les détours inutiles et la stabilité du comportement.


## 9. Comment généraliser à des agents réels

Dans ce projet, l’agent Taxi-v3 apprend à optimiser une politique dans un environnement simple et parfaitement connu.
La même logique d'instrumentation (logs d'épisodes, définition de KPI, comparaison de politiques) peut s'appliquer à des agents déployés en entreprise :

- agent de support client : mesurer taux de résolution, satisfaction, temps moyen de prise en charge ;
- agent logistique : mesurer respect des contraintes, coût total, délais de livraison ;
- agent de recommandation : mesurer clics, conversion, rétention.

Ce mini-environnement sert donc de sandbox pour expérimenter des pratiques d’“Agent Ops” (observabilité, métriques, amélioration itérative) avant de les appliquer à des systèmes d’agents plus complexes, comme ceux présentés dans le 5-Day AI Agents Intensive (multi-agents, outils, mémoire, A/B testing, etc.).
